In [1]:
from pyspark.sql import SparkSession

# Criação da sessão Spark
spark = SparkSession.builder.appName("Analise de Hábitos dos Alunos").getOrCreate()

# Leitura do arquivo CSV
df = spark.read.option("header", "true").option("inferSchema", "true").csv("E:/Mestrado UFCG/kaggle-habitos-dos-alunos-desempenho/dado/StudentPerformance.csv")

# Exibindo as primeiras 5 linhas
df.show(5)


+----------+---+------+-------------------+------------------+-------------+-------------+---------------------+-----------+------------+------------------+------------------------+----------------+--------------------+-----------------------------+----------+
|student_id|age|gender|study_hours_per_day|social_media_hours|netflix_hours|part_time_job|attendance_percentage|sleep_hours|diet_quality|exercise_frequency|parental_education_level|internet_quality|mental_health_rating|extracurricular_participation|exam_score|
+----------+---+------+-------------------+------------------+-------------+-------------+---------------------+-----------+------------+------------------+------------------------+----------------+--------------------+-----------------------------+----------+
|     S1000| 23|Female|                0.0|               1.2|          1.1|           No|                 85.0|        8.0|        Fair|                 6|                  Master|         Average|                   